In [2]:
import numpy as np
import pandas as pd

In [3]:
simRAO = 2000
nMTCD = 30000
MTCD_data = pd.read_csv(f'MTCD_data_{nMTCD}.csv', index_col=False)

In [4]:
data = np.zeros((simRAO, 4), dtype=int)
Preamble_status = pd.DataFrame(
    data,
    columns=['nRA', 'empty', 'collided', 'success'],
)
Preamble_status.index.name = 'system frame'

In [5]:
print(Preamble_status.shape)
Preamble_status.sample(5)

(2000, 4)


,nRA,empty,collided,success
system frame,,,,
94,0,0,0,0
817,0,0,0,0
1785,0,0,0,0
687,0,0,0,0
1158,0,0,0,0


In [6]:
RA_data = MTCD_data[['RA_init', 'RA_first', 'RA_success', 'RA_transmit']]
RA_data.index.name = 'device_id'

In [7]:
print(RA_data.shape)
RA_data.sample(5)

(30000, 4)


,RA_init,RA_first,RA_success,RA_transmit
device_id,,,,
22433,434,434,-1,0
13986,549,549,-1,0
26406,833,833,-1,0
27686,304,304,-1,0
19006,661,661,-1,0


In [8]:
D2D_member = pd.DataFrame(
    columns=['group', 'Member', 'arrange', 'request', 'delay', 'transmit'],
)

# assign group
D2D_member['group'] = MTCD_data['clusters']

# assign type
D2D_member['Member'] = (D2D_member.index != D2D_member['group'])

# assign arrange
arrangePRB = 20
for group in D2D_member['group'].unique():
    members = D2D_member.loc[(D2D_member['group'] == group) &
                             (D2D_member['Member'])]
    n = 0
    for device_id, member in members.iterrows():
        D2D_member.at[device_id, 'arrange'] = n // arrangePRB
        n += 1

# initiate request
D2D_member['request'] = np.zeros(nMTCD, dtype=bool)

# initiate delay
D2D_member['delay'] = np.zeros(nMTCD, dtype=int)

# initiate transmit
D2D_member['transmit'] = np.zeros(nMTCD, dtype=int)

In [9]:
print(D2D_member.shape)
D2D_member.loc[D2D_member['Member'] == False].sample(5)

(30000, 6)


,group,Member,arrange,request,delay,transmit
device_id,,,,,,
143,143,False,NaN,False,0,0
947,947,False,NaN,False,0,0
655,655,False,NaN,False,0,0
1380,1380,False,NaN,False,0,0
57,57,False,NaN,False,0,0


In [10]:
D2D_group = pd.DataFrame(
    columns=['Header', 'class', 'N_member', 'RA_request', 'arrange', 'HL']
)

In [11]:
D2D_group['Header'] = MTCD_data['clusters'].unique()

In [12]:
D2D_group['class'] = [x // 54 for x in range(len(D2D_group))]

In [14]:
D2D_group

,Header,class,N_member,RA_request,arrange,HL
0,0,0,NaN,NaN,NaN,NaN
1,1,0,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN
4,4,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
214,1004,3,NaN,NaN,NaN,NaN
215,1056,3,NaN,NaN,NaN,NaN
216,1756,4,NaN,NaN,NaN,NaN
217,9805,4,NaN,NaN,NaN,NaN


In [18]:
BS_schedule = pd.DataFrame(
    columns=['class', 'TDCR'],
)

# assign class of every frame
BS_schedule['class'] = [f % N_class for f in range(simRAO)]

# initialize TDCR
BS_schedule['TDCR'] = np.zeros(simRAO, dtype=int)

In [19]:
BS_schedule

,class,TDCR
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
1995,0,0
1996,1,0
1997,2,0
1998,3,0


In [39]:
schedule = BS_schedule.loc[np.arange(0,16)]
schedule

,class,TDCR
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,0,0
6,1,0
7,2,0
8,3,0
9,4,0


In [52]:
s = schedule.loc[(schedule['class'] == 1)]
s

,class,TDCR
1,1,0
6,1,0
11,1,0


In [58]:
s.iat[0, 1] = 1
s

,class,TDCR
1,1,1
6,1,0
11,1,0


In [61]:
s.index[0]

1

In [64]:
BS_schedule.at[s.index[0], 'TDCR'] = 1235

In [65]:
BS_schedule

,class,TDCR
0,0,0
1,1,1235
2,2,0
3,3,0
4,4,0
...,...,...
1995,0,0
1996,1,0
1997,2,0
1998,3,0
